<a href="https://colab.research.google.com/github/dhu16/fortune_sentiments/blob/main/Week8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install PyDrive

In [12]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [13]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [36]:
%matplotlib inline
import pandas as pd
import string
import nltk
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14)

In [ ]:
downloaded = drive.CreateFile({'id':"1O6RaEYB3Pu8y1IyLbJPZTSRTN9ekgN7-"})
downloaded.GetContentFile('fortune_phrases.xlsx')

In [ ]:
#Create dataframe with only 1 column containing the sentences
df = pd.read_excel('fortune_phrases.xlsx', usecols=['phrase'])

#Convert dataframe to list
content=df.stack().tolist()
content[0:10]

In [ ]:
#Using nltk for stop words
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
def full_remove(x, removal_list):
    for w in removal_list:
        x = x.replace(w, ' ')
    return x

#Remove punctuation
removePunc = [full_remove(x, list(string.punctuation)) for x in content]

#Make everything lower-case and remove any white space
sents_lower = [x.lower() for x in removePunc]
sents_lower = [x.strip() for x in sents_lower]

#Remove stop words
stops = stopwords.words("english")

def removeStopWords(stopWords, txt):
    newtxt = ' '.join([word for word in txt.split() if word not in stopWords])
    return newtxt

stop_set = ['the', 'a', 'an', 'i', 'he', 'she', 'they', 'to', 'of', 'it', 'from']
    
processedSentences = [removeStopWords(stop_set,x) for x in sents_lower]
processedSentences[0:20]

In [73]:
#Using TextBlob to determine polarity and subjectivity of sentences
from textblob import TextBlob

#Create polarity function and subjectivity function
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

#Finds all words that can be assinged polarity and subjectivity and averages them together for the whole sentence's respective polarity/subjectivity
pol_list = [pol(x) for x in processedSentences]
sub_list = [sub(x) for x in processedSentences]

In [ ]:
#Append polarity and subjectivity sentiments to dataframe
df['Polarity']=pol_list
df['Subjectivity']=sub_list

#Print dataframe
pd.set_option('display.max_colwidth', -1)
df.head(10)